In [1]:
import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import MultiPolygon
from shapely.geometry import LineString
from shapely.geometry import MultiPoint
import json
import os 
import time
import gzip
from folium.plugins import MarkerCluster

In [21]:
estaciones = pd.read_csv("estaciones-metrobus.csv")
rutas = gpd.read_file("metrobus-rutas.geojson")
semaforos = gpd.read_file("semaforos.geojson")
ruta_linea_1 = rutas.iloc[2,1]
print(ruta_linea_1)

LINESTRING (-99.1445379871106 19.46196472772886, -99.14430211887945 19.46219404778804, -99.1440430576165 19.46246945322279, -99.14349933061864 19.46300328828099, -99.143109100838 19.46340930462186, -99.14284934490942 19.46371726278168, -99.14176996693396 19.46502583558683, -99.1411666950755 19.46576469346224, -99.14101232566033 19.46597272275545, -99.14084205162973 19.46624444831915, -99.14059069851562 19.46667230652222, -99.14048268345283 19.46683173912723, -99.14036718003568 19.46697779826272, -99.1396965772941 19.46779371546262, -99.1389727373505 19.46866379959403, -99.13815394872742 19.46966624419978, -99.13783849803521 19.47004842901673, -99.13748606877616 19.47042657492038, -99.13701957165947 19.47094582610406, -99.13687667012547 19.47112337492209, -99.13664434919409 19.47141063751749, -99.13636923705113 19.47178151609289, -99.13610486869159 19.47213307456766, -99.1358699025855 19.47243124155617, -99.13519132900937 19.47326095788538, -99.1350023237824 19.47348985109671, -99.13460

In [3]:
columns = estaciones.columns[0].split(";")

In [4]:
estaciones["1"] = "1"
estaciones["2"] = "1"
estaciones.columns = columns


In [5]:
estaciones["Nombre"] = estaciones["wkt_geom"].str.split(";").str[1]

In [6]:
estaciones["Linea"] = estaciones["wkt_geom"].str.split(";").str[2]

In [7]:
estaciones["wkt_geom"] = estaciones["wkt_geom"].str.split(";").str[0]

In [8]:
estaciones["wkt_geom"] = estaciones["wkt_geom"].str.extract(r'(-\d{2}\.\d*\s\d{2}\.\d*)', expand=False)
estaciones_linea_1 = estaciones[estaciones["Linea"] == "Línea 1"]

In [9]:
mc = MarkerCluster()

map = folium.Map(location=[23.6260333, -102.5375005], zoom_start=5, tiles="CartoDB dark_matter")
for point in estaciones_linea_1["wkt_geom"]:
    point1 = point.split(" ")[0]
    point2 = point.split(" ")[1]
    points = list((float(point2),float(point1)))
    mc.add_child(folium.CircleMarker(location=points))
    map.add_child(mc)

In [10]:
map

In [18]:
folium.GeoJson(rutas['geometry']).add_to(map)
map


RuntimeError: b'no arguments in initialization list'

In [12]:
map.save("linea_1_metrobus.html")

In [13]:
semaforos_linea_1 = []
for point in semaforos["geometry"]:
    coord = []
    if ruta_linea_1.distance(point) < 1e-4:
        point_x = point.x
        point_y = point.y
        coord.append(point_y)
        coord.append(point_x)
        semaforos_linea_1.append(coord)
print(len(semaforos_linea_1))

40


In [14]:

map2 = folium.Map(location=[23.6260333, -102.5375005], zoom_start=5, tiles="CartoDB dark_matter")
for semaforo in semaforos_linea_1:
    folium.CircleMarker(semaforo).add_to(map2)

folium.GeoJson(ruta_linea_1).add_to(map2)
map2

In [15]:
map2.save("semaforos_linea_1_metrobus.html")